In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import distance
from sklearn.impute import KNNImputer
import inshallah as ins
from xgboost import XGBRegressor

done


In [29]:
# lmao.to_csv("cleaned.csv", sep="|")
lmao = pd.read_csv("cleaned.csv", sep="|")
lmao["time"] = pd.to_datetime(lmao["time"])

vessels = pd.read_csv("../CSV/vessels.csv", sep="|")
""" imputer = KNNImputer(n_neighbors=10)
non_num = vessels[["vesselId", "shippingLineId", "homePort"]]
vessels.drop(columns=["vesselId", "shippingLineId", "homePort"], inplace=True)

cols, ind = vessels.columns, vessels.index

vessels = imputer.fit_transform(vessels)
vessels = pd.DataFrame(vessels, columns=cols, index=ind)

vessels["vesselId"] = non_num["vesselId"]
vessels["shippingLineId"] = non_num["shippingLineId"]
vessels["homePort"] = non_num["homePort"]
 """
lmao["vesselId"] = pd.Categorical(lmao["vesselId"], categories=lmao["vesselId"].unique())
lmao["shippingLineId"] = pd.Categorical(lmao["shippingLineId"], categories=lmao["shippingLineId"].unique())

test = pd.read_csv("../CSV/ais_test.csv")

test["time"] = pd.to_datetime(test["time"])
test.drop(columns=["scaling_factor", "ID"], inplace=True)

lmao["NT"].head()


0    19051.8
1    19091.0
2    18987.7
3    18506.7
4    18159.0
Name: NT, dtype: float64

In [3]:
stats = {}

for ids in lmao["vesselId"].unique():
    vessel_stats = {}

    speed = lmao[lmao["vesselId"] == ids]["speed_from_prev"].mean() / 3.6
    vessel_stats["avg_speed"] = speed
    # vessel_stats["avg_time_at_sea"] = lmao[lmao["vesselId"] == ids]["time_at_sea"].mean()

    stats[ids] = vessel_stats

print(stats)


{'61e9f3a8b937134a3c4bfdf7': {'avg_speed': 2.5190297470815417}, '61e9f3d4b937134a3c4bff1f': {'avg_speed': 2.0536094133209812}, '61e9f436b937134a3c4c0131': {'avg_speed': 3.897379720942899}, '61e9f3b4b937134a3c4bfe77': {'avg_speed': 3.3259447664086865}, '61e9f41bb937134a3c4c0087': {'avg_speed': 2.835339773430691}, '61e9f468b937134a3c4c028f': {'avg_speed': 4.486639091692623}, '61e9f46bb937134a3c4c02b3': {'avg_speed': 5.612883964615154}, '61e9f3bfb937134a3c4bfe9f': {'avg_speed': 4.082324280135849}, '61e9f45bb937134a3c4c0221': {'avg_speed': 2.2916122575980022}, '61e9f3e6b937134a3c4bff6d': {'avg_speed': 2.1961863944225604}, '61e9f3c9b937134a3c4bfef5': {'avg_speed': 2.8338160942383785}, 'clh6aqawa0002gh0zypfa5dut': {'avg_speed': 3.322574139703564}, '61e9f3adb937134a3c4bfe39': {'avg_speed': 2.4449826110664747}, '61e9f3b1b937134a3c4bfe55': {'avg_speed': 2.1223251713049907}, '61e9f3c6b937134a3c4bfecd': {'avg_speed': 3.757123431415154}, '61e9f3c6b937134a3c4bfed5': {'avg_speed': 0.0006690800436548

In [23]:
n = 5

ninja = test["vesselId"].unique()[0:n]
lmao = lmao[lmao["vesselId"].isin(test["vesselId"].unique()[0:2*n])]

X_test_pre = []

for ids in ninja:
    boat = test[test["vesselId"] == ids]
    boat_train = lmao[lmao["vesselId"] == ids]

    boat.sort_values("time", inplace=True)
    boat.reset_index(drop=True, inplace=True)

    #Distance stuff
    boat["delta_time"] = boat["time"] - boat["time"].shift(1)
    boat.at[0, "delta_time"] = (boat.at[0, "time"] - pd.to_datetime(boat_train.iloc[-1]["time"]))
    boat["delta_time"] = boat["delta_time"].dt.total_seconds()

    boat["dist_from_prev"] = (boat["delta_time"] * stats[ids]["avg_speed"]) / 1000

    #Time at sea
    last = lmao[lmao["vesselId"] == ids].iloc[-1]["time_at_sea"]
    boat["time_cum"] = boat["delta_time"].cumsum()
    boat["time_at_sea"] = boat["time_cum"] + last

    for _,row in boat.iterrows():
        X_test_pre.append(row.to_dict())

X_test_pre = pd.DataFrame(X_test_pre).sort_values("time")
X_test_pre = pd.merge(X_test_pre, vessels, how="left")

X_test_pre["vesselId"] = pd.Categorical(X_test_pre["vesselId"], categories=lmao["vesselId"].unique())
X_test_pre["shippingLineId"] = pd.Categorical(X_test_pre["shippingLineId"], categories=lmao["shippingLineId"].unique())

#Time stuff
origin = pd.to_datetime("2024-01-01 00:00:00")

lmao["time"] = (lmao["time"] - origin).dt.total_seconds()
X_test_pre["time"] = (X_test_pre["time"] - origin).dt.total_seconds()

print(type(lmao["time"].iloc[0]))
print(type(X_test_pre["time"].iloc[0]))


<class 'numpy.float64'>
<class 'numpy.float64'>


In [33]:
X = lmao[["time", "vesselId", "length", "dist_from_prev"]]
y = lmao[["delta_lat", "delta_lon"]]

X_test = X_test_pre[["time", "vesselId", "length", "dist_from_prev"]]

#Standardizing
X_mean = X[["length", "dist_from_prev"]].mean()
X_std = X[["length", "dist_from_prev"]].std()

X[["length", "dist_from_prev"]] = (X[["length", "dist_from_prev"]] - X_mean) / X_std

""" X_mean_test = X_test[["NT", "dist_from_prev"]].mean()
X_std_test = X_test[["NT", "dist_from_prev"]].std()

X_test[["NT", "dist_from_prev"]] = (X_test[["NT", "dist_from_prev"]] - X_mean_test) / X_std_test """

#1-Hot
X = pd.get_dummies(X, "vesselId")
X_test = pd.get_dummies(X_test, "vesselId")

# --- XGBOOST ---
xgb = XGBRegressor(verbose=True, n_estimators = 50)
xgb.fit(X, y)
y_pred = xgb.predict(X_test)

y_pred = pd.DataFrame(y_pred)
y_pred.head()

# --- KNN ---
""" from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor

knn = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=5))
knn.fit(X, y)
y_pred = knn.predict(X_test) """

,length
0,230.0
1,124.0
2,186.0
3,183.0
4,182.0


In [65]:
y_pred = pd.DataFrame(y_pred)
y_pred.head()

NameError: name 'y_pred' is not defined

In [8]:
y_pred["vesselId"] = X_test_pre["vesselId"]
y_pred["time"] = X_test_pre["time"]

res = []

for i in y_pred["vesselId"].unique():
    boat = y_pred[y_pred["vesselId"] == i]

    last_lon = lmao[lmao["vesselId"] == i].iloc[-1]["longitude"]
    last_lat = lmao[lmao["vesselId"] == i].iloc[-1]["latitude"]

    boat["lat_cum"] = boat[0].cumsum()
    boat["lon_cum"] = boat[1].cumsum()

    boat["longitude_predicted"] = boat["lon_cum"] + last_lon
    boat["latitude_predicted"] = boat["lat_cum"] + last_lat

    for _,row in boat.iterrows():
        res.append(row.to_dict())

res_df = pd.DataFrame(res)

""" boat_pred = y_pred.copy()
boat_train = boat_train[["vesselId", "time", "latitude", "longitude"]]

boat_pred["vesselId"] = vessel
boat_pred["time"] = boat_time

boat_pred["lat_cum"] = y_pred[0].cumsum()
boat_pred["lon_cum"] = y_pred[1].cumsum()

last_lat = boat_train.iloc[-1]["latitude"]
last_lon = boat_train.iloc[-1]["longitude"]

boat_pred["latitude"] = boat_pred["lat_cum"] + last_lat
boat_pred["longitude"] = boat_pred["lon_cum"] + last_lon

boat_pred = boat_pred[["vesselId", "time", "latitude", "longitude"]]

boat_train.head() """
res_df.sort_values("time", inplace=True)
res_df.reset_index(drop=True, inplace=True)
res_df.head()

,0,1,vesselId,time,lat_cum,lon_cum,longitude_predicted,latitude_predicted
0,0.002138,0.000082,61e9f3aeb937134a3c4bfe3d,2024-05-08 00:03:16,0.002138,0.000082,-81.497808,31.148608
1,-0.000082,0.000212,61e9f473b937134a3c4c02df,2024-05-08 00:06:17,-0.000082,0.000212,120.296462,14.816858
2,-0.004478,0.005414,61e9f469b937134a3c4c029b,2024-05-08 00:10:02,-0.004478,0.005414,10.788214,38.274472
3,0.000248,-0.000176,61e9f45bb937134a3c4c0221,2024-05-08 00:10:34,0.000248,-0.000176,172.835044,-43.537602
4,-0.013060,-0.015364,61e9f38eb937134a3c4bfd8d,2024-05-08 00:12:27,-0.013060,-0.015364,-6.135394,48.520140


In [17]:
res_df = res_df[["longitude_predicted", "latitude_predicted"]]
res_df["longitude_predicted"].clip(lower=-180, upper=180, inplace=True)
res_df["latitude_predicted"].clip(lower=-90, upper=90, inplace=True)
res_df.to_csv("inshallah.csv")

In [26]:
vessel = "61e9f3aeb937134a3c4bfe3d"

boat_train = lmao[lmao["vesselId"] == vessel][["vesselId", "time", "latitude", "longitude"]]
boat_pred = res_df[res_df["vesselId"] == vessel] [["vesselId", "time", "latitude_predicted", "longitude_predicted"]]

boat_pred["latitude"] = boat_pred["latitude_predicted"]
boat_pred["longitude"] = boat_pred["longitude_predicted"]

boat_pred.drop(columns=["latitude_predicted", "longitude_predicted"], inplace=True)

boat_pred.head()

,vesselId,time,latitude,longitude
0,61e9f3aeb937134a3c4bfe3d,2024-05-08 00:03:16,31.148608,-81.497808
5,61e9f3aeb937134a3c4bfe3d,2024-05-08 00:36:14,31.148636,-81.497672
9,61e9f3aeb937134a3c4bfe3d,2024-05-08 00:51:12,31.149098,-81.500044
12,61e9f3aeb937134a3c4bfe3d,2024-05-08 01:12:14,31.153314,-81.504282
15,61e9f3aeb937134a3c4bfe3d,2024-05-08 01:30:14,31.153114,-81.503950


In [27]:
fig = (
    ins.visualize_vessel_movements(boat_train)
)
fig.show()

fig = (
    ins.visualize_vessel_movements(boat_pred)
)
fig.show()